In [64]:
!pip install -q langchain
!pip install -q transfomers sentencepiece accelerate bitsandbytes
!pip install -q python-dotenv
!pip install langchain-google-genai
!pip install google-colab
!pip install langchain langchain-community
!pip install -q unstructured
!pip install nltk
!pip install -q wikipedia
!pip install -q pypdf
!pip install -q tiktoken

ERROR: Could not find a version that satisfies the requirement transfomers (from versions: none)
ERROR: No matching distribution found for transfomers


In [96]:
from dotenv import dotenv_values
from langchain.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage
from langchain_community.chat_models import ChatGooglePalm
import os
import pandas as pd
from langchain.document_loaders import DataFrameLoader
from langchain.document_loaders import UnstructuredURLLoader
from langchain.document_loaders import WikipediaLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import OnlinePDFLoader
from langchain.document_loaders import DirectoryLoader
from transformers import AutoTokenizer
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter



env_values = dotenv_values("./app.env")

google_api_key = env_values['GOOGLE_API_KEY']

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7,
    google_api_key=google_api_key
)

In [66]:
!wget -O youtube-sub.csv https://raw.githubusercontent.com/Petlja/JupyterBookSrCyr/master/podaci/Top%2025%20YouTubers.csv

--2025-06-16 15:07:21--  https://raw.githubusercontent.com/Petlja/JupyterBookSrCyr/master/podaci/Top%2025%20YouTubers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1153 (1.1K) [text/plain]
Saving to: ‘youtube-sub.csv’

youtube-sub.csv     100%[===================>]   1.13K  --.-KB/s    in 0s      

2025-06-16 15:07:21 (64.2 MB/s) - ‘youtube-sub.csv’ saved [1153/1153]



In [67]:
df=pd.read_csv("youtube-sub.csv")
df

,RANK,GRADE,NAME,VIDEOS,SUBSCRIBERS,VIEWES
0,1,A++,T-Series,13629,105783888,76945588449
1,2,A,PewDiePie,3898,97853589,22298927681
2,3,A+,5-Minute Crafts,3341,58629572,14860695079
3,4,A++,Cocomelon - Nursery Rhymes,441,53163816,33519273951
4,5,A++,SET India,31923,51784081,36464793233
5,6,A+,Canal KondZilla,1100,50560964,25446405744
6,7,A+,WWE,42404,46098586,34085586984
7,8,B+,Justin Bieber,134,45873439,625649566
8,9,A,Dude Perfect,209,43796634,8354321862
9,10,A+,Badabun,4406,41131131,13175713909


In [68]:
df=df[["NAME","VIDEOS","SUBSCRIBERS","VIEWES"]]
df

,NAME,VIDEOS,SUBSCRIBERS,VIEWES
0,T-Series,13629,105783888,76945588449
1,PewDiePie,3898,97853589,22298927681
2,5-Minute Crafts,3341,58629572,14860695079
3,Cocomelon - Nursery Rhymes,441,53163816,33519273951
4,SET India,31923,51784081,36464793233
5,Canal KondZilla,1100,50560964,25446405744
6,WWE,42404,46098586,34085586984
7,Justin Bieber,134,45873439,625649566
8,Dude Perfect,209,43796634,8354321862
9,Badabun,4406,41131131,13175713909


In [69]:
llm_loader=DataFrameLoader(df,page_content_column="NAME")
docs=llm_loader.load()
docs

[Document(metadata={'VIDEOS': 13629, 'SUBSCRIBERS': 105783888, 'VIEWES': 76945588449}, page_content='T-Series'),
 Document(metadata={'VIDEOS': 3898, 'SUBSCRIBERS': 97853589, 'VIEWES': 22298927681}, page_content='PewDiePie'),
 Document(metadata={'VIDEOS': 3341, 'SUBSCRIBERS': 58629572, 'VIEWES': 14860695079}, page_content='5-Minute Crafts'),
 Document(metadata={'VIDEOS': 441, 'SUBSCRIBERS': 53163816, 'VIEWES': 33519273951}, page_content='Cocomelon - Nursery Rhymes'),
 Document(metadata={'VIDEOS': 31923, 'SUBSCRIBERS': 51784081, 'VIEWES': 36464793233}, page_content='SET India'),
 Document(metadata={'VIDEOS': 1100, 'SUBSCRIBERS': 50560964, 'VIEWES': 25446405744}, page_content='Canal KondZilla'),
 Document(metadata={'VIDEOS': 42404, 'SUBSCRIBERS': 46098586, 'VIEWES': 34085586984}, page_content='WWE'),
 Document(metadata={'VIDEOS': 134, 'SUBSCRIBERS': 45873439, 'VIEWES': 625649566}, page_content='Justin Bieber'),
 Document(metadata={'VIDEOS': 209, 'SUBSCRIBERS': 43796634, 'VIEWES': 83543218

In [70]:
urls = [
    "https://www.apple.com/in/support/products/faqs.html",
    "https://www.apple.com/legal/sales-support/",
]
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()
data

[Document(metadata={'source': 'https://www.apple.com/in/support/products/faqs.html'}, page_content='Frequently Asked Questions\n\nWhere can I purchase the AppleCare Protection Plan?\n\nThe AppleCare Protection Plan is available at the Apple Online Store and many Apple-authorised resellers and wireless service providers.\n\nHow do I initiate repair service under the AppleCare Protection Plan?\n\nCarry-in service. Carry your product into an Apple Authorised Service Provider.\n\nOnsite service. Contact us and we’ll help you arrange an Apple-authorised repair for your desktop Mac at your location, at no additional charge.\n\nDo-It-Yourself service. Contact us and we may be able to send you what you need to service your own product, such as accessories.\n\nThe AppleCare Protection Plan for Mac, for iPod and for Apple TV provide global repair coverage. Service will be limited to the options available in the country where service is requested. Service options, parts availability and response 

In [71]:
query="Nikola Tesla"
llm_data=WikipediaLoader(query=query,load_max_docs=3,doc_content_chars_max=20000)
data=llm_data.load()
data

[Document(metadata={'title': 'Nikola Tesla', 'summary': "Nikola Tesla (; Serbian Cyrillic: Никола Тесла [nǐkola têsla]; 10 July 1856 – 7 January 1943) was a Serbian-American engineer, futurist, and inventor. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.\nBorn and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gained practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884 he immigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His AC induction motor and related polyphase AC patents, l

In [72]:
len(data)

3

In [73]:
data[0].page_content

'Nikola Tesla (; Serbian Cyrillic: Никола Тесла [nǐkola têsla]; 10 July 1856 – 7 January 1943) was a Serbian-American engineer, futurist, and inventor. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.\nBorn and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gained practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884 he immigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His AC induction motor and related polyphase AC patents, licensed by Westinghouse Electric in 1888, earned him a c

In [74]:
data[0].metadata

{'title': 'Nikola Tesla',
 'summary': "Nikola Tesla (; Serbian Cyrillic: Никола Тесла [nǐkola têsla]; 10 July 1856 – 7 January 1943) was a Serbian-American engineer, futurist, and inventor. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.\nBorn and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gained practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884 he immigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His AC induction motor and related polyphase AC patents, licensed by Westing

In [75]:
!wget -O laravel.pdf https://laraveldaily.com/wp-content/uploads/2020/04/laravel-tips-2020-04.pdf

--2025-06-16 15:07:32--  https://laraveldaily.com/wp-content/uploads/2020/04/laravel-tips-2020-04.pdf
Resolving laraveldaily.com (laraveldaily.com)... 172.67.216.144, 104.21.86.73, 2606:4700:3036::6815:5649, ...
Connecting to laraveldaily.com (laraveldaily.com)|172.67.216.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 314570 (307K) [application/pdf]
Saving to: ‘laravel.pdf’

laravel.pdf         100%[===================>] 307.20K  --.-KB/s    in 0.03s   

2025-06-16 15:07:32 (8.85 MB/s) - ‘laravel.pdf’ saved [314570/314570]



In [76]:
llm_loader=PyPDFLoader("./laravel.pdf")
pages=llm_loader.load_and_split()
pages[1]

Document(metadata={'producer': 'Skia/PDF m84', 'creator': 'PyPDF', 'creationdate': '', 'source': './laravel.pdf', 'total_pages': 41, 'page': 1, 'page_label': '2'}, page_content='History of changes\n \nApril 20, 2020: \n\u200b \n40 more tips, total 100 now (removed duplicated, updated some old ones)\n \nApril 28, 2019: \n\u200b \n10 more tips, total 60 now\n \nNovember 8, 2018\n\u200b \n: 10 more tips, total 50 now\n \nOctober 9, 2018\n\u200b \n: Book release with 40 tips\n \n \nTip 1. Single Action Controllers\n \n \nIf you want to create a controller with just one action, you can use \n\u200b \n__invoke()\n\u200b \n method and\n \neven create "invokable" controller.\n \n \n<?php\n \nnamespace App\\Http\\Controllers;\n \n \nuse App\\User;\n \nuse App\\Http\\Controllers\\Controller;\n \n \nclass ShowProfile extends Controller\n \n{\n \n    /**\n \n     * Show the profile for the given user.\n \n     *\n \n     * @param  int  $id\n \n     * @return Response\n \n     */\n \n    public fun

In [77]:
pages[1].page_content

'History of changes\n \nApril 20, 2020: \n\u200b \n40 more tips, total 100 now (removed duplicated, updated some old ones)\n \nApril 28, 2019: \n\u200b \n10 more tips, total 60 now\n \nNovember 8, 2018\n\u200b \n: 10 more tips, total 50 now\n \nOctober 9, 2018\n\u200b \n: Book release with 40 tips\n \n \nTip 1. Single Action Controllers\n \n \nIf you want to create a controller with just one action, you can use \n\u200b \n__invoke()\n\u200b \n method and\n \neven create "invokable" controller.\n \n \n<?php\n \nnamespace App\\Http\\Controllers;\n \n \nuse App\\User;\n \nuse App\\Http\\Controllers\\Controller;\n \n \nclass ShowProfile extends Controller\n \n{\n \n    /**\n \n     * Show the profile for the given user.\n \n     *\n \n     * @param  int  $id\n \n     * @return Response\n \n     */\n \n    public function __invoke($id)\n \n    {\n \n        return view(\'user.profile\', [\'user\' => User::findOrFail($id)]);\n \n    }\n \n}\n \nRoutes:\n \nRoute::get(\'user/{id}\', \'ShowPro

In [78]:
pages[1].metadata

{'producer': 'Skia/PDF m84',
 'creator': 'PyPDF',
 'creationdate': '',
 'source': './laravel.pdf',
 'total_pages': 41,
 'page': 1,
 'page_label': '2'}

In [79]:
len(pages)

41

Text spliters

In [80]:
query1="Nikola Tesla"
query2="Thomas Edison"

doc1=WikipediaLoader(query=query1,load_max_docs=3,doc_content_chars_max=20000).load()
doc2=WikipediaLoader(query=query2,load_max_docs=3,doc_content_chars_max=20000).load()
text_documents1=doc1[0].page_content
text_documents2=doc2[0].page_content

In [81]:
print(text_documents1)

Nikola Tesla (; Serbian Cyrillic: Никола Тесла [nǐkola têsla]; 10 July 1856 – 7 January 1943) was a Serbian-American engineer, futurist, and inventor. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.
Born and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gained practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884 he immigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His AC induction motor and related polyphase AC patents, licensed by Westinghouse Electric in 1888, earned him a con

In [82]:
print(text_documents2)

Thomas Alva Edison (February 11, 1847 – October 18, 1931) was an American inventor and businessman. He developed many devices in fields such as electric power generation, mass communication, sound recording, and motion pictures. These inventions, which include the phonograph, the motion picture camera, and early versions of the electric light bulb, have had a widespread impact on the modern industrialized world. He was one of the first inventors to apply the principles of organized science and teamwork to the process of invention, working with many researchers and employees. He established the first industrial research laboratory.
Edison was raised in the American Midwest. Early in his career he worked as a telegraph operator, which inspired some of his earliest inventions. In 1876, he established his first laboratory facility in Menlo Park, New Jersey, where many of his early inventions were developed. He later established a botanical laboratory in Fort Myers, Florida, in collaboratio

characters text splitter

In [83]:
documents=[text_documents1,text_documents2]
metadata=[{"document":query1},{"document":query2}]
text_splitter=CharacterTextSplitter(separator="",chunk_size=300,chunk_overlap=40,length_function=len)
normal_chunks=text_splitter.create_documents(documents,metadatas=metadata)

In [84]:
print(text_documents1)

Nikola Tesla (; Serbian Cyrillic: Никола Тесла [nǐkola têsla]; 10 July 1856 – 7 January 1943) was a Serbian-American engineer, futurist, and inventor. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.
Born and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gained practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884 he immigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His AC induction motor and related polyphase AC patents, licensed by Westinghouse Electric in 1888, earned him a con

In [85]:
normal_chunks[1]

Document(metadata={'document': 'Nikola Tesla'}, page_content='tem.\nBorn and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without receiving a degree. He then gained practical experience in the early 1880s working in telephony and at Continental Edison in the new electric power industry. In 1884 he immigrated to the Uni')

In [86]:
normal_chunks[2]

Document(metadata={'document': 'Nikola Tesla'}, page_content='dustry. In 1884 he immigrated to the United States, where he became a naturalized citizen. He worked for a short time at the Edison Machine Works in New York City before he struck out on his own. With the help of partners to finance and market his ideas, Tesla set up laboratories and companies in Ne')

In [87]:
normal_chunks[3].page_content

'set up laboratories and companies in New York to develop a range of electrical and mechanical devices. His AC induction motor and related polyphase AC patents, licensed by Westinghouse Electric in 1888, earned him a considerable amount of money and became the cornerstone of the polyphase system whi'

In [88]:
normal_chunks[3].metadata

{'document': 'Nikola Tesla'}

In [89]:
len(normal_chunks)

154

NLTK

In [90]:
import nltk
from langchain.text_splitter import NLTKTextSplitter
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [91]:
text_splitter=NLTKTextSplitter(chunk_size=500)
nltk_chunks=text_splitter.create_documents(documents,metadatas=metadata)

In [92]:
len(nltk_chunks)

131

In [93]:
nltk_chunks[5].page_content

'Tesla became well known as an inventor and demonstrated his achievements to celebrities and wealthy patrons at his lab, and was noted for his showmanship at public lectures.\n\nThroughout the 1890s, Tesla pursued his ideas for wireless lighting and worldwide wireless electric power distribution in his high-voltage, high-frequency power experiments in New York and Colorado Springs.\n\nIn 1893, he made pronouncements on the possibility of wireless communication with his devices.'

In [94]:
nltk_chunks[5].metadata

{'document': 'Nikola Tesla'}

Tokens | Transformers Tokens

In [97]:
model_id="stabilityai/stablelm-tuned-alpha-3b"
toknizer=AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [101]:
text_splitter=CharacterTextSplitter.from_huggingface_tokenizer(toknizer,chunk_size=100,chunk_overlap=20)
tokens_chunks=text_splitter.create_documents(documents,metadatas=metadata)
tokens_chunks = text_splitter.create_documents(
    documents, metadatas=metadata
)

In [102]:
tokens_chunks[5].page_content

'=== Working at Budapest Telephone Exchange ===\nTesla moved to Budapest, Hungary, in 1881 to work under Tivadar Puskás at a telegraph company, the Budapest Telephone Exchange. Upon arrival, Tesla realized that the company, then under construction, was not functional, so he worked as a draftsman in the Central Telegraph Office instead. Within a few months, the Budapest Telephone Exchange became functional, and Tesla was allocated the chief electrician position. During his employment, Tesla made many improvements to the Central Station equipment and claimed to have perfected a telephone repeater or amplifier, which was never patented nor publicly described.'

In [103]:
 tokens_chunks[5].metadata

{'document': 'Nikola Tesla'}